## Доклад. Выбор наиболее важных для настройки модели признаков посредством алгоритма Gain Ratio

Для начала, определим, что такое ***Gain Ratio***
$$Gain\ Ratio(S,A)=\frac{Information\ Gain(S,A)}{Split\ Information(A)}$$
Information Gain показывает уменьшение энтропии после  разбиения данных по признаку:
$$Information\ Gain(S,A) = Entropy(S)-\sum\limits_{v}\frac{|S_v|}{|S|} Entropy(S_v) $$
Split Information - информация о разбиении
$$Split\ Information(A) = -\sum\limits_v \frac{|S_v|}{|S|}  \log_2 \frac{|S_v|}{|S|} $$
Энтропия - мера беспорядочности
$$Entropy(S) = - \sum\limits_i p_i \log_2 p_i$$


Далее, для работы с кодом нужно загрузить датасет. В моём случае - это дефолт клиентов по кредитным картам

Импортируем библиотеки

In [2]:
import pandas as pd
import numpy as np

Загрузим датасет

In [ ]:
credit_defaults = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/UCI_Credit_Card.csv')

Отобразим первые несколько строк датасета, для наглядности

In [ ]:
print(credit_defaults.head())

   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1    20000.0    2          2         1   24      2      2     -1     -1   
1   2   120000.0    2          2         2   26     -1      2      0      0   
2   3    90000.0    2          2         2   34      0      0      0      0   
3   4    50000.0    2          2         1   37      0      0      0      0   
4   5    50000.0    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...        0.0        0.0        0.0       0.0     689.0       0.0   
1  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
2  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
3  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
4  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default.payment.next.month  
0       0.0       0.0   

Тут можно видеть все возможные значения всех признаков

In [ ]:
print("Названия признаков:")
print(credit_defaults.columns.tolist())

print("\nУникальные значения для каждого признака:")
for column in credit_defaults.columns:
    unique_values = credit_defaults[column].unique()
    print(f"{column}: {unique_values}")

Названия признаков:
['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'default.payment.next.month']

Уникальные значения для каждого признака:
ID: [    1     2     3 ... 29998 29999 30000]
LIMIT_BAL: [  20000.  120000.   90000.   50000.  500000.  100000.  140000.  200000.
  260000.  630000.   70000.  250000.  320000.  360000.  180000.  130000.
  450000.   60000.  230000.  160000.  280000.   10000.   40000.  210000.
  150000.  380000.  310000.  400000.   80000.  290000.  340000.  300000.
   30000.  240000.  470000.  480000.  350000.  330000.  110000.  420000.
  170000.  370000.  270000.  220000.  190000.  510000.  460000.  440000.
  410000.  490000.  390000.  580000.  600000.  620000.  610000.  700000.
  670000.  680000.  430000.  550000.  540000. 1000000.  530000.  710000.
  5

Теперь посчитаем характеристики для каждого признака.

***ЭНТРОПИЯ***

In [ ]:
def entropy(y):
  counts = np.bincount(y) # Подсчитываем количество вхождений элементов массива
  probabilities = counts / len(y) # Считаем вероятность вхождения каждого элемента
  return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

***INFORMATION GAIN***

In [ ]:
# X - датасет, y - классификационный признак, fearture_idx - индекс другого признака
def information_gain(X, y, feature_idx):
  total_entropy = entropy(y) # Энтропия до разбиения на признаки
  values, counts = np.unique(X[:, feature_idx], return_counts=True)
  weighted_entropy = 0 # Энтропия после разбиения на признаки

  for value, count in zip(values, counts):
      subset_y = y[X[:, feature_idx] == value]
      weighted_entropy += (count / len(X)) * entropy(subset_y)

  return total_entropy - weighted_entropy

***SPLIT INFORMATION***

In [ ]:
def split_information(X, feature_idx):
    values, counts = np.unique(X[:, feature_idx], return_counts=True)
    probabilities = counts / len(X)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

***GAIN RATIO***

In [ ]:
def gain_ratio(X, y, feature_idx):
    ig = information_gain(X, y, feature_idx)
    iv = split_information(X, feature_idx)
    return ig / iv if iv != 0 else 0

Теперь применим все эти функции для расчёта коэффициента усиления для каждого из признаков датасета(за исключением классфикационного, разумеется).

In [ ]:
# Исключаем из датасета ненужные колонки
X = credit_defaults.drop(['ID', 'default.payment.next.month'], axis=1)

#Отделим классфикационный признак
y = credit_defaults['default.payment.next.month'].values

In [ ]:
feature_names = X.columns.tolist()
X = X.values
scores = [gain_ratio(X, y, i) for i in range(X.shape[1])]

# Ранжирование признаков
feature_ranking = np.argsort(scores)[::-1]
print("Gain Ratio:")
for idx in feature_ranking:
    print(f"{feature_names[idx]}: {scores[idx]:.4f}")

Gain Ratio:
PAY_0: 0.0529
BILL_AMT1: 0.0429
BILL_AMT2: 0.0428
BILL_AMT3: 0.0428
BILL_AMT4: 0.0425
BILL_AMT5: 0.0419
BILL_AMT6: 0.0418
PAY_2: 0.0384
PAY_3: 0.0295
PAY_4: 0.0268
PAY_5: 0.0255
PAY_6: 0.0216
PAY_AMT1: 0.0215
PAY_AMT3: 0.0209
PAY_AMT6: 0.0206
PAY_AMT2: 0.0205
PAY_AMT5: 0.0200
PAY_AMT4: 0.0199
LIMIT_BAL: 0.0047
EDUCATION: 0.0028
SEX: 0.0012
MARRIAGE: 0.0008
AGE: 0.0007


Но есть нюанс: многие признаки являются числовыми, а значит, что и значений у них много. Их можно дискретизировать(например, на 5 интервалов), чтобы лучше изменить результат

In [1]:
X_discretized = np.zeros_like(X)
for i in range(X.shape[1]):
    X_discretized[:, i] = pd.cut(X[:, i], bins=5, labels=False, duplicates='drop')

NameError: name 'np' is not defined

In [ ]:
scores = [gain_ratio(X_discretized, y, i) for i in range(X_discretized.shape[1])]

# Ранжирование признаков
feature_ranking = np.argsort(scores)[::-1]
print("\nРейтинг важности признаков:")
for idx in feature_ranking:
    print(f"{feature_names[idx]}: {scores[idx]:.4f}")


Рейтинг важности признаков:
PAY_0: 0.1034
PAY_2: 0.1015
PAY_5: 0.0823
PAY_3: 0.0809
PAY_4: 0.0796
PAY_6: 0.0701
PAY_AMT1: 0.0119
LIMIT_BAL: 0.0103
PAY_AMT4: 0.0101
PAY_AMT2: 0.0075
PAY_AMT5: 0.0058
PAY_AMT6: 0.0053
EDUCATION: 0.0027
PAY_AMT3: 0.0023
SEX: 0.0012
BILL_AMT2: 0.0009
BILL_AMT1: 0.0008
MARRIAGE: 0.0008
BILL_AMT5: 0.0005
BILL_AMT4: 0.0004
AGE: 0.0004
BILL_AMT6: 0.0003
BILL_AMT3: 0.0001


А теперь можно взглянуть на библиотечную реализацию выбора k самых важных признаков и провести сравнение.

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

X = credit_defaults.drop(['ID', 'default.payment.next.month'], axis=1)
y = credit_defaults['default.payment.next.month']

selector = SelectKBest(score_func = f_classif, k = 10)
X_best = selector.fit_transform(X, y)
best_indices = selector.get_support(indices=True)

selected_features = X.columns[best_indices]
print(selected_features.tolist())



['LIMIT_BAL', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT4']


In [ ]:

colors = np.array([1,1,2,3,3])
fruits = np.array([1,2,2,3,4])

MyArr = np.array([
  [1,1,1],
  [1,2,2],
  [2,2,2],
  [3,1,3],
  [4,2,4]
])

print(f"Entropy(fruits) = {entropy(fruits)}")
print(f"IG(Fruit, Color) = {information_gain(MyArr, fruits, 0)}")
print(f"SI(Color) = {split_information(MyArr, 0)}")
print(f"GR(Color) = {gain_ratio(MyArr, fruits, 0)}")

print(f"IG(Fruit, Size) = {information_gain(MyArr, fruits, 1)}")
print(f"SI(Size) = {split_information(MyArr, 1)}")
print(f"GR(Size) = {gain_ratio(MyArr, fruits, 1)}")


print(f"GR(Colors) = {gain_ratio(MyArr, colors, 2)}")



Entropy(fruits) = 1.9219280948873623
IG(Fruit, Color) = 1.5219280948873624
SI(Color) = 1.9219280948873623
GR(Color) = 0.7918756684685216
IG(Fruit, Size) = 0.9709505944546686
SI(Size) = 0.9709505944546686
GR(Size) = 1.0
GR(Colors) = 0.5837513369370432
